# Gaussian Mixture Models:

* The simplicity of k-means leads to practical challenges in its applications.
    * In particular, the nonprobabilistic nature of k-means and its use of simple distance from cluster center to assign cluster membership leads to poor performance for many real-world situations.

* Gaussian Mixture Models:
    * Can be viewed as an extension of the ideas of k-means, but can also be powerful tool for estimation beyond simple clustering.


In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

# A. Weaknesses of k-Means: